In [151]:
%load_ext autoreload
%autoreload 2
from functools import partial
import itertools
import collections
import pandas as pd
import numpy as np
import uproot
import awkward as ak
import awkward_pandas as akpd
import sys
sys.path.append("..")
from hists.dataframe import *

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
input_file = '/data_cms_upgrade/cuisset/testbeam18/clue3d/v29/single-file/data/CLUE_clusters.root'
entryStart = 0
entryStop = 100

In [119]:
comp = DataframeComputations(uproot.open(input_file + ":clusters").arrays(entry_start=entryStart, entry_stop=entryStop))

In [4]:
event = 98

In [5]:
comp.clusters3D.loc[event]

,beamEnergy,clus3D_x,clus3D_y,clus3D_z,clus3D_energy,clus3D_size
clus3D_id,,,,,,
0,20.0,2.542433,-0.854914,21.770227,16.609020,21
1,20.0,2.428030,1.530388,31.837212,1.323122,8


In [6]:
px.scatter_3d(comp.clusters3D.loc[event], x="clus3D_x", y="clus3D_y", z="clus3D_z", size="clus3D_size", color="beamEnergy").show("json")

In [224]:
fig = go.Figure(
    layout=go.Layout(
        title=go.layout.Title(text="Event visualizer"),
        width=1200,
        height=600
    )
)
camera = dict(
    eye=dict(x=0., y=0., z=2.5)
)
fig.update_layout(scene_camera=camera)
fig.update_layout(legend=dict(groupclick="togglegroup"))

df = comp.clusters3D.loc[event]

color_cycle = itertools.cycle(px.colors.qualitative.Plotly)
mapClus3Did_color = {clus3D_id : next(color_cycle) for clus3D_id in df.index.get_level_values(0).drop_duplicates().to_list()}

fig.add_trace(go.Scatter3d(
    name="3D tracksters",
    x=df["clus3D_x"], y=df["clus3D_y"], z=df["clus3D_z"], 
    mode="markers",
    marker=dict(
        symbol="cross",
        color=list(mapClus3Did_color.values()),
        size=df["clus3D_size"]*2,
    ),
    hovertemplate="clus3D_x=%{x}<br>clus3D_y=%{y}<br>clus3D_z=%{z}<br>clus3D_size=%{marker.size}<extra></extra>",
    )
)
None

In [225]:
df = (comp
    .clusters3D_merged_2D_custom(comp.clusters3D_with_clus2D_id, comp.clusters2D_withNearestHigher)
    .set_index(["event", "clus3D_id", "clus2D_id"])
    .loc[event]
)
df = df.join(df[["clus2D_x", "clus2D_y", "clus2D_z"]], on=["clus3D_id", "clus2D_nearestHigher"], rsuffix="_ofNearestHigher")
df

beamEnergy  ...  clus2D_z_ofNearestHigher
clus3D_id clus2D_id              ...                          
0         0                20.0  ...                 17.672501
          1                20.0  ...                 17.672501
          2                20.0  ...                 17.672501
          3                20.0  ...                 20.577499
          4                20.0  ...                       NaN
          5                20.0  ...                 19.687500
          6                20.0  ...                 19.687500
          7                20.0  ...                 20.577499
          8                20.0  ...                 22.692499
          9                20.0  ...                 20.577499
          10               20.0  ...                 28.702499
          11               20.0  ...                 29.592501
          12               20.0  ...                 23.582500
          14               20.0  ...                 28.702499
          17               20.0  ...                 29.592501
          20               20.0  ...                 32.397499
          21               20.0  ...                 38.007500
          24               20.0  ...                 32.397499
          26               20.0  ...                 34.312500
          28               20.0  ...                 38.007500
          29               20.0  ...                 39.922501
1         15               20.0  ...                 31.507500
          16               20.0  ...                       NaN
          18               20.0  ...                 35.202499
          19               20.0  ...                 31.507500
          22               20.0  ...                 31.507500
          23               20.0  ...                 34.312500
          25               20.0  ...                 35.202499
          27               20.0  ...                 35.202499

[29 rows x 17 columns]

In [226]:
grouped_df = df.groupby("clus3D_id").get_group(0)
grouped_df

beamEnergy  ...  clus2D_z_ofNearestHigher
clus3D_id clus2D_id              ...                          
0         0                20.0  ...                 17.672501
          1                20.0  ...                 17.672501
          2                20.0  ...                 17.672501
          3                20.0  ...                 20.577499
          4                20.0  ...                       NaN
          5                20.0  ...                 19.687500
          6                20.0  ...                 19.687500
          7                20.0  ...                 20.577499
          8                20.0  ...                 22.692499
          9                20.0  ...                 20.577499
          10               20.0  ...                 28.702499
          11               20.0  ...                 29.592501
          12               20.0  ...                 23.582500
          14               20.0  ...                 28.702499
          17               20.0  ...                 29.592501
          20               20.0  ...                 32.397499
          21               20.0  ...                 38.007500
          24               20.0  ...                 32.397499
          26               20.0  ...                 34.312500
          28               20.0  ...                 38.007500
          29               20.0  ...                 39.922501

[21 rows x 17 columns]

In [227]:
def makeArrow(x1, x2, y1, y2, z1, z2, dictLine=dict(), dictCone=dict()):
    traces = []
    try:
        lengthFactor = 1./math.sqrt((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)
        traces.append(go.Scatter3d(
            **collections.ChainMap(dictLine, dict( # Give preference to keywords from arguments over those specified here
                mode="lines",
                hoverinfo='skip',
                x=[x1, x2],
                y=[y1, y2],
                z=[z1, z2]
            ))
        ))
        traces.append(go.Cone(
            **collections.ChainMap(dictCone, dict(
                x=[x2], y=[y2], z=[z2],
                u=[lengthFactor*(x2-x1)],
                v=[lengthFactor*(y2-y1)],
                w=[lengthFactor*(z2-z1)],
                colorscale=[[0, "blue"], [1, "blue"]],
                showscale=False,
                anchor="tip",
                hoverinfo="skip"
            ))
        ))
    except ZeroDivisionError: # In case two rechits are in the same point (should not happen in theory, but...)
        pass
    return traces

In [228]:
showLegend = True
for clus3D_id, grouped_df in df.groupby("clus3D_id"):
    fig.add_trace(go.Scatter3d(
        mode="markers",
        legendgroup="cluster2D",
        legendgrouptitle_text="2D clusters",
        name=f"Trackster nb {clus3D_id}",
        x=grouped_df["clus2D_x"], y=grouped_df["clus2D_y"], z=grouped_df["clus2D_z"], 
        marker=dict(
            symbol="circle",
            color=mapClus3Did_color[clus3D_id],
            size=grouped_df["clus2D_size"]
        ),
        customdata=np.dstack((grouped_df.clus2D_energy, grouped_df.clus2D_rho, grouped_df.clus2D_delta,
            grouped_df.clus2D_pointType.map({0:"Follower", 1:"Seed", 2:"Outlier"})))[0],
        #hovertemplate="clus2D_x=%{x}<br>clus2D_y=%{y}<br>clus2D_z=%{z}<br>clus2D_size=%{marker.size}<extra></extra>",
        hovertemplate=(
            "2D cluster : %{customdata[3]}<br>"
            "Energy: %{customdata[0]:.2g} GeV<br>Rho: %{customdata[1]:.2g} GeV<br>"
            "Delta: %{customdata[2]:.2g} cm<br>"
            "Size: %{marker.size}"
        )
    ))

    for row in grouped_df.dropna().itertuples(index=False, name="Cluster2D"):
        fig.add_traces(makeArrow(
            row.clus2D_x, row.clus2D_x_ofNearestHigher, row.clus2D_y, row.clus2D_y_ofNearestHigher, row.clus2D_z, row.clus2D_z_ofNearestHigher,
            dictLine=dict(
                name="Cluster 2D chain",
                legendgroup="clus2D_chain",
                showlegend=showLegend
            ), dictCone=dict(
                legendgroup="clus2D_chain"
            )
            )
        )

        showLegend = False


In [229]:
df_rechits = (comp
    .clusters3D_merged_rechits_custom(["rechits_x", "rechits_y", "rechits_z", "rechits_energy", 
        "rechits_rho", "rechits_delta", "rechits_nearestHigher", "rechits_pointType"])
    .loc[event]
    .reset_index(level=["rechits_layer", "rechits_id"])
    .set_index("clus2D_id", append=True)
)
df_rechits = df_rechits.join(df_rechits[["rechits_id", "rechits_x", "rechits_y", "rechits_z"]].set_index("rechits_id"), on=["rechits_nearestHigher"], rsuffix="_ofNearestHigher")
df_rechits

rechits_layer  ...  rechits_z_ofNearestHigher
clus3D_id clus2D_id                 ...                           
0         0                      1  ...                    13.8775
          0                      1  ...                    13.8775
          0                      1  ...                        NaN
          1                      2  ...                    14.7675
          1                      2  ...                    14.7675
...                            ...  ...                        ...
1         23                    17  ...                    37.1175
          25                    17  ...                    37.1175
          25                    17  ...                    37.1175
          27                    18  ...                        NaN
          27                    18  ...                    38.0075

[252 rows x 20 columns]

In [230]:
df_rechits.query("rechits_x == rechits_x_ofNearestHigher and rechits_y == rechits_y_ofNearestHigher ")

rechits_layer  ...  rechits_z_ofNearestHigher
clus3D_id clus2D_id                 ...                           
0         2                      3  ...                  16.782499
          4                      5  ...                  19.687500
          6                      7  ...                  22.692499
          9                      9  ...                  25.697500

[4 rows x 20 columns]

In [231]:
color_cycle = itertools.cycle(px.colors.qualitative.Plotly)
mapClus2Did_color = {clus2D_id : next(color_cycle) for clus2D_id in df.index.get_level_values("clus2D_id").drop_duplicates().to_list()}

showLegend = True
for index, grouped_df in df_rechits.groupby(level=["clus3D_id", "clus2D_id"]):
    fig.add_trace(go.Scatter3d(
        mode="markers",
        legendgroup="rechits",
        legendgrouptitle_text="Rechits",
        name=f"2D cluster nb {index[1]}",
        x=grouped_df["rechits_x"], y=grouped_df["rechits_y"], z=grouped_df["rechits_z"], 
        marker=dict(
            symbol="circle",
            color=mapClus3Did_color[clus3D_id],
            size=grouped_df["rechits_energy"]*30
        ),
        customdata=np.dstack((grouped_df.rechits_energy, grouped_df.rechits_rho, grouped_df.rechits_delta,
            grouped_df.rechits_pointType.map({0:"Follower", 1:"Seed", 2:"Outlier"})))[0],
        #hovertemplate="clus2D_x=%{x}<br>clus2D_y=%{y}<br>clus2D_z=%{z}<br>clus2D_size=%{marker.size}<extra></extra>",
        hovertemplate=(
            "Rechit : %{customdata[3]}<br>"
            "Energy: %{customdata[0]:.2g} GeV<br>Rho: %{customdata[1]:.2g} GeV<br>"
            "Delta: %{customdata[2]:.2g} cm"
        )
    ))
    for row in grouped_df.dropna().itertuples(index=False):
        fig.add_traces(makeArrow(
            row.rechits_x, row.rechits_x_ofNearestHigher, row.rechits_y, row.rechits_y_ofNearestHigher, row.rechits_z, row.rechits_z_ofNearestHigher,
            dictLine=dict(
                name="Rechits chain",
                legendgroup="rechits_chain",
                showlegend=showLegend
            ), dictCone=dict(
                legendgroup="rechits_chain"
            )
            )
        )
        showLegend = False
        

In [232]:

fig.show()

In [207]:
comp.rechits.loc[event].loc[25:26]

,beamEnergy,rechits_x,rechits_y,rechits_z,rechits_energy,rechits_layer,rechits_rho,rechits_delta,rechits_nearestHigher,rechits_pointType
rechits_id,,,,,,,,,,
25,20.0,1.732805,0.685358,16.782499,0.024450,3,0.310677,0.000,26,0
26,20.0,1.732805,0.685358,16.782499,0.071707,3,0.334306,1.125,17,0


In [235]:
comp.rechits.loc[event]

,beamEnergy,rechits_x,rechits_y,rechits_z,rechits_energy,rechits_layer,rechits_rho,rechits_delta,rechits_nearestHigher,rechits_pointType
rechits_id,,,,,,,,,,
0,20.0,3.626411,-1.587286,13.877500,0.021062,1,0.061886,1.125173e+00,1,0
1,20.0,2.651933,-1.024786,13.877500,0.081648,1,0.194167,1.125303e+00,2,0
2,20.0,1.677445,-0.462042,13.877500,0.203976,1,0.244800,3.402823e+38,-1,1
3,20.0,1.677445,-2.712539,13.877500,0.200998,1,0.200998,2.250497e+00,2,1
4,20.0,0.703080,1.225458,13.877500,0.014758,1,0.014758,1.948601e+00,2,1
...,...,...,...,...,...,...,...,...,...,...
279,20.0,5.925315,-3.654544,49.681999,0.047627,25,0.047627,3.402823e+38,-1,1
280,20.0,3.981021,1.907357,52.881500,0.022236,27,0.022236,1.911437e+00,282,1
281,20.0,3.981021,-0.343132,52.881500,0.015394,27,0.033082,1.094088e+00,282,0


In [236]:
comp.array[98]

<Record {beamEnergy: 20, ntupleNumber: 441, ...} type='{beamEnergy: float32...'>